# Wynn Resort Chatbot Agent Tutorial
## Part 3: Building the Conversational AI Agent

This notebook demonstrates the AI agent that understands Vegas hotel context and converts natural language into forecast adjustments.

In [ ]:
import os
from datetime import datetime, timedelta
import anthropic
import json
import re
from dateutil import parser
from dotenv import load_dotenv

## Step 1: Set up Anthropic API

In [ ]:
# Load environment variables from .env file
load_dotenv()

# See .env.example for the format you should use in your .env file
# For your own Anthropic API key, go to Anthropic and register an account then you will possibly need to buy $5 of Anthropic credits.

True

## Step 2: Build the Forecast Agent

In [ ]:
class WynnForecastAgent:
    def __init__(self):
        self.client = anthropic.Anthropic(api_key=os.environ.get('ANTHROPIC_API_KEY'))
        self.conversation_history = []
    
    def get_next_monday(self):
        """Helper to find next Monday"""
        today = datetime.now().date()
        days_ahead = 0 - today.weekday()
        if days_ahead <= 0:
            days_ahead += 7
        return today + timedelta(days_ahead)
    
    def parse_date_reference(self, date_str, base_date=None):
        """Parse natural language date references"""
        if not base_date:
            base_date = datetime.now().date()
        
        date_str = date_str.lower()
        
        if "tomorrow" in date_str:
            return base_date + timedelta(days=1)
        elif "next week" in date_str or "next monday" in date_str:
            return self.get_next_monday()
        elif "this weekend" in date_str:
            days_until_friday = (4 - base_date.weekday()) % 7
            if days_until_friday == 0 and base_date.weekday() == 4:
                return base_date
            return base_date + timedelta(days=days_until_friday)
        elif "fight night" in date_str or "fight weekend" in date_str:
            # Typically Saturday
            days_until_saturday = (5 - base_date.weekday()) % 7
            if days_until_saturday == 0:
                days_until_saturday = 7
            return base_date + timedelta(days=days_until_saturday)
        
        # Default to tomorrow
        return base_date + timedelta(days=1)
    
    def process_message(self, message):
        """Process user message and return adjustments"""
        self.conversation_history.append({"role": "user", "content": message})
        
        prompt = f"""You are an AI assistant for Wynn Resort Las Vegas operations forecasting. 
Analyze the user's message about hotel operations and extract any forecast modifications for:
- rooms (occupancy percentage)
- cleaning (staff needed)
- security (staff needed)

User message: "{message}"
Today's date: {datetime.now().date()}
Current time: {datetime.now().strftime('%H:%M')}
Forecast period: Next 168 hours (7 days)

Extract modifications and return JSON:
{{
    "response": "Natural conversational response acknowledging the changes",
    "modifications": [
        {{
            "metric": "rooms|cleaning|security",
            "type": "percentage|absolute|set",
            "value": number,
            "start_date": "YYYY-MM-DD HH:MM",
            "end_date": "YYYY-MM-DD HH:MM",
            "reason": "brief explanation"
        }}
    ]
}}

Vegas-specific examples:
- "Big fight this Saturday" → Increase rooms to 95%+, add security
- "Convention checking in Monday morning" → Increase cleaning staff 10am-2pm
- "Pool party season starting" → More cleaning staff afternoons
- "Construction on the Strip" → Might reduce walk-in traffic

Consider hourly patterns:
- Check-in rush: 3-6 PM
- Check-out rush: 10 AM-12 PM
- Casino security peaks: 10 PM-3 AM

If just chatting (no modifications), return empty modifications array but still respond naturally."""

        try:
            response = self.client.messages.create(
                model="claude-3-5-sonnet-20241022",
                max_tokens=1000,
                messages=[
                    {"role": "user", "content": prompt}
                ]
            )
            
            text = response.content[0].text
            
            # Find JSON in response
            json_match = re.search(r'\{[\s\S]*\}', text)
            if json_match:
                result = json.loads(json_match.group())
                self.conversation_history.append({"role": "assistant", "content": result['response']})
                return result
            
        except Exception as e:
            print(f"Agent error: {e}")
        
        # Fallback response
        fallback = {
            "response": "I understand you're asking about resort operations. Could you be more specific about what changes you'd like to make to the forecast?",
            "modifications": []
        }
        self.conversation_history.append({"role": "assistant", "content": fallback['response']})
        return fallback

# Create the agent
agent = WynnForecastAgent()
print("Wynn Forecast Agent initialized!")

Wynn Forecast Agent initialized!


## Step 3: Test Conversations

In [ ]:
# Helper function to display conversations nicely
def chat_with_agent(message):
    print(f"\n👤 You: {message}")
    result = agent.process_message(message)
    print(f"\n🤖 Agent: {result['response']}")
    
    if result['modifications']:
        print("\n📊 Forecast Adjustments:")
        for mod in result['modifications']:
            print(f"   - {mod.get('metric', 'N/A')}: {mod.get('type', 'N/A')} "
                  f"by {mod.get('value', 'N/A')} from {mod.get('start_date', 'N/A')} "
                  f"to {mod.get('end_date', 'N/A')}")
            if 'reason' in mod:
                print(f"     Reason: {mod['reason']}")
    
    return result

In [ ]:
# Test 1: Basic greeting
chat_with_agent("Hello! How are you doing today?")


👤 You: Hello! How are you doing today?
Agent error: Error code: 401 - {'type': 'error', 'error': {'type': 'authentication_error', 'message': 'invalid x-api-key'}}

🤖 Agent: I understand you're asking about resort operations. Could you be more specific about what changes you'd like to make to the forecast?


{'response': "I understand you're asking about resort operations. Could you be more specific about what changes you'd like to make to the forecast?",
 'modifications': []}

In [ ]:
# Test 2: Event-based adjustment
chat_with_agent("There's a big UFC fight happening this Saturday night")


👤 You: There's a big UFC fight happening this Saturday night
Agent error: Error code: 401 - {'type': 'error', 'error': {'type': 'authentication_error', 'message': 'invalid x-api-key'}}

🤖 Agent: I understand you're asking about resort operations. Could you be more specific about what changes you'd like to make to the forecast?


{'response': "I understand you're asking about resort operations. Could you be more specific about what changes you'd like to make to the forecast?",
 'modifications': []}

In [ ]:
# Test 3: Convention adjustment
chat_with_agent("We have a tech convention with 5000 attendees checking in Monday morning")


👤 You: We have a tech convention with 5000 attendees checking in Monday morning
Agent error: Error code: 401 - {'type': 'error', 'error': {'type': 'authentication_error', 'message': 'invalid x-api-key'}}

🤖 Agent: I understand you're asking about resort operations. Could you be more specific about what changes you'd like to make to the forecast?


{'response': "I understand you're asking about resort operations. Could you be more specific about what changes you'd like to make to the forecast?",
 'modifications': []}

In [ ]:
# Test 4: Multiple metrics
chat_with_agent("Pool party season is starting this weekend. Expecting younger crowds and messier rooms.")


👤 You: Pool party season is starting this weekend. Expecting younger crowds and messier rooms.
Agent error: Error code: 401 - {'type': 'error', 'error': {'type': 'authentication_error', 'message': 'invalid x-api-key'}}

🤖 Agent: I understand you're asking about resort operations. Could you be more specific about what changes you'd like to make to the forecast?


{'response': "I understand you're asking about resort operations. Could you be more specific about what changes you'd like to make to the forecast?",
 'modifications': []}

In [ ]:
# Test 5: Security concern
chat_with_agent("Our competitor had some incidents last night. Better increase security for the weekend.")


👤 You: Our competitor had some incidents last night. Better increase security for the weekend.
Agent error: Error code: 401 - {'type': 'error', 'error': {'type': 'authentication_error', 'message': 'invalid x-api-key'}}

🤖 Agent: I understand you're asking about resort operations. Could you be more specific about what changes you'd like to make to the forecast?


{'response': "I understand you're asking about resort operations. Could you be more specific about what changes you'd like to make to the forecast?",
 'modifications': []}

## Step 4: Simulate a Full Conversation

In [ ]:
# Have a multi-turn conversation
print("=== Full Conversation Demo ===")

chat_with_agent("Hi! I'm the operations manager. Can you help me adjust our staffing forecast?")
chat_with_agent("We just got word that there's a huge boxing match this Saturday")
chat_with_agent("Also, the pool deck construction finished early, so we can open it tomorrow")
chat_with_agent("What changes have we made so far?")

=== Full Conversation Demo ===

👤 You: Hi! I'm the operations manager. Can you help me adjust our staffing forecast?
Agent error: Error code: 401 - {'type': 'error', 'error': {'type': 'authentication_error', 'message': 'invalid x-api-key'}}

🤖 Agent: I understand you're asking about resort operations. Could you be more specific about what changes you'd like to make to the forecast?

👤 You: We just got word that there's a huge boxing match this Saturday
Agent error: Error code: 401 - {'type': 'error', 'error': {'type': 'authentication_error', 'message': 'invalid x-api-key'}}

🤖 Agent: I understand you're asking about resort operations. Could you be more specific about what changes you'd like to make to the forecast?

👤 You: Also, the pool deck construction finished early, so we can open it tomorrow
Agent error: Error code: 401 - {'type': 'error', 'error': {'type': 'authentication_error', 'message': 'invalid x-api-key'}}

🤖 Agent: I understand you're asking about resort operations. Could

{'response': "I understand you're asking about resort operations. Could you be more specific about what changes you'd like to make to the forecast?",
 'modifications': []}

## Step 5: Export Agent Logic for Backend Integration

In [ ]:
# Save the agent class to a file for later use
agent_code = '''
import os
from datetime import datetime, timedelta
import anthropic
import json
import re

class WynnForecastAgent:
    """AI Agent for Wynn Resort forecast adjustments"""
    
    def __init__(self):
        self.client = anthropic.Anthropic(api_key=os.environ.get('ANTHROPIC_API_KEY'))
        self.conversation_history = []
    
    # ... (rest of the class implementation)
'''

with open('wynn_forecast_agent.py', 'w') as f:
    f.write("# Wynn Resort Forecast Agent\n")
    f.write("# This file contains the conversational AI logic\n\n")
    f.write("# Copy the WynnForecastAgent class from the notebook here\n")

print("Agent code template saved to wynn_forecast_agent.py")
print("\nNext steps:")
print("1. Copy the full WynnForecastAgent class to the file")
print("2. Integrate with WebSocket backend")
print("3. Connect forecast adjustments to the RNN model")

Agent code template saved to wynn_forecast_agent.py

Next steps:
1. Copy the full WynnForecastAgent class to the file
2. Integrate with WebSocket backend
3. Connect forecast adjustments to the RNN model


## Summary

We've built a conversational AI agent that:
1. Understands Vegas hotel context
2. Parses natural language into structured adjustments
3. Maintains conversation history
4. Handles various scenarios (fights, conventions, security)

The agent is ready to be integrated with our forecast model and WebSocket backend!